# Análise de Dados – Controle Concentrado de Constitucionalidade (STF)

Notebook interativo para explorar os dados extraídos do portal do STF (ADIs, ADPFs, ADCs e ADOs).

In [ ]:
import pandas as pd
import json
import ast
import re
from pathlib import Path

## 1. Carregar Dados

In [ ]:
CSV_PATH = "ArquivosConcatenados_1.csv"

if not Path(CSV_PATH).exists():
    CSV_PATH = "Dados ADI de 6000 a 6010.csv"
    print(f"ArquivosConcatenados_1.csv não encontrado, usando {CSV_PATH}")

df = pd.read_csv(CSV_PATH)
print(f"Arquivo: {CSV_PATH}")
print(f"Linhas: {len(df):,} | Colunas: {df.shape[1]}")

In [ ]:
df.head(3)

In [ ]:
df.dtypes

In [ ]:
df.describe(include="all")

In [ ]:
df.isnull().sum().sort_values(ascending=False)

## 2. Pré-processamento

In [ ]:
df["data_protocolo"] = pd.to_datetime(df["data_protocolo"], format="%d/%m/%Y", errors="coerce")
df["ano"] = df["data_protocolo"].dt.year
df["decada"] = (df["ano"] // 10 * 10).astype("Int64")
df["tem_liminar"] = df["liminar"].str.contains("MEDIDA LIMINAR", na=False)

UF_NAMES = {
    "AC": "Acre", "AL": "Alagoas", "AP": "Amapá", "AM": "Amazonas",
    "BA": "Bahia", "CE": "Ceará", "DF": "Distrito Federal", "ES": "Espírito Santo",
    "GO": "Goiás", "MA": "Maranhão", "MT": "Mato Grosso", "MS": "Mato Grosso do Sul",
    "MG": "Minas Gerais", "PA": "Pará", "PB": "Paraíba", "PR": "Paraná",
    "PE": "Pernambuco", "PI": "Piauí", "RJ": "Rio de Janeiro",
    "RN": "Rio Grande do Norte", "RS": "Rio Grande do Sul", "RO": "Rondônia",
    "RR": "Roraima", "SC": "Santa Catarina", "SP": "São Paulo", "SE": "Sergipe",
    "TO": "Tocantins",
}
df["origem_valida"] = df["origem"].where(df["origem"].isin(UF_NAMES))
df["estado_nome"] = df["origem_valida"].map(UF_NAMES)

print(f"Período: {df['ano'].min():.0f} – {df['ano'].max():.0f}")
print(f"Classes: {df['classe'].unique().tolist()}")

## 3. Parsing de Colunas Complexas

As colunas `partes_total`, `andamentos_lista`, `decisões` e `deslocamentos_lista` contêm listas/dicts serializados como strings. Use as funções abaixo para convertê-las.

In [ ]:
def parse_json_col(series: pd.Series) -> pd.Series:
    """Parse a column containing JSON-encoded strings."""
    def _try(val):
        if pd.isna(val):
            return []
        try:
            return json.loads(val)
        except (json.JSONDecodeError, TypeError):
            try:
                return ast.literal_eval(val)
            except Exception:
                return []
    return series.apply(_try)


def parse_list_col(series: pd.Series) -> pd.Series:
    """Parse a column containing Python list literals."""
    def _try(val):
        if pd.isna(val):
            return []
        try:
            return ast.literal_eval(val)
        except Exception:
            return []
    return series.apply(_try)

In [ ]:
# Exemplo: parsear andamentos de um processo específico
# (cuidado: parsear a coluna inteira é lento para ~6600 linhas)

sample = df.iloc[0]
andamentos = json.loads(sample["andamentos_lista"]) if pd.notna(sample["andamentos_lista"]) else []
print(f"Processo: {sample['nome_processo']}")
print(f"Andamentos: {len(andamentos)}")
if andamentos:
    print(f"Primeiro: {andamentos[0]}")

## 4. Exploração Básica

In [ ]:
df["classe"].value_counts()

In [ ]:
df["status_processo"].value_counts()

In [ ]:
df["relator"].value_counts().head(15)

In [ ]:
df["origem_valida"].value_counts().head(10)

In [ ]:
df.groupby("classe")[["len(andamentos_lista)", "len(decisões)", "len(partes_total)", "len(deslocamentos)"]].describe()

## 5. Filtros e Consultas

Modifique as células abaixo para filtrar os dados conforme sua necessidade.

In [ ]:
# Filtrar por classe
adis = df[df["classe"] == "ADI"]
print(f"ADIs: {len(adis):,}")
adis.head()

In [ ]:
# Filtrar por status
em_andamento = df[df["status_processo"] == "Em andamento"]
print(f"Em andamento: {len(em_andamento):,}")
em_andamento.head()

In [ ]:
# Filtrar por relator
relator = "GILMAR MENDES"  # altere conforme desejado
df_relator = df[df["relator"] == relator]
print(f"Processos de {relator}: {len(df_relator):,}")
df_relator[["nome_processo", "classe", "data_protocolo", "status_processo"]].head(10)

In [ ]:
# Filtrar por período
recentes = df[df["ano"] >= 2020]
print(f"Processos de 2020 em diante: {len(recentes):,}")
recentes.groupby(["ano", "classe"]).size().unstack(fill_value=0)

In [ ]:
# Buscar processo por nome
busca = "6000"  # altere conforme desejado
resultado = df[df["nome_processo"].str.contains(busca, case=False, na=False)]
print(f"Resultados para '{busca}': {len(resultado)}")
resultado[["nome_processo", "classe", "relator", "autor1", "status_processo"]]

## 6. Análise de Andamentos (por processo)

Como as colunas de andamentos e partes são muito grandes, é melhor trabalhar processo a processo.

In [ ]:
def get_andamentos(processo_nome: str) -> pd.DataFrame:
    """Retorna os andamentos de um processo como DataFrame."""
    row = df[df["nome_processo"] == processo_nome].iloc[0]
    raw = row["andamentos_lista"]
    try:
        items = json.loads(raw)
    except Exception:
        items = ast.literal_eval(raw) if pd.notna(raw) else []
    return pd.DataFrame(items)


def get_partes(processo_nome: str) -> pd.DataFrame:
    """Retorna as partes de um processo como DataFrame."""
    row = df[df["nome_processo"] == processo_nome].iloc[0]
    raw = row["partes_total"]
    try:
        items = json.loads(raw)
    except Exception:
        items = ast.literal_eval(raw) if pd.notna(raw) else []
    return pd.DataFrame(items)


def get_decisoes(processo_nome: str) -> pd.DataFrame:
    """Retorna as decisões de um processo como DataFrame."""
    row = df[df["nome_processo"] == processo_nome].iloc[0]
    raw = row["decisões"]
    try:
        items = json.loads(raw)
    except Exception:
        items = ast.literal_eval(raw) if pd.notna(raw) else []
    return pd.DataFrame(items)

In [ ]:
# Exemplo: andamentos do primeiro processo
processo = df["nome_processo"].iloc[0]
and_df = get_andamentos(processo)
print(f"Andamentos de {processo}: {len(and_df)}")
and_df.head(10)

## 7. Exportar Subconjuntos

Use as células abaixo para salvar filtros ou transformações em novos arquivos.

In [ ]:
# Exportar colunas leves (sem andamentos/partes/decisões) para Excel
cols_leves = [
    "incidente", "classe", "nome_processo", "classe_extenso",
    "tipo_processo", "liminar", "origem", "relator", "autor1",
    "len(partes_total)", "data_protocolo", "origem_orgao",
    "lista_assuntos", "len(andamentos_lista)", "len(decisões)",
    "len(deslocamentos)", "status_processo",
]

# df[cols_leves].to_csv("dados_leves.csv", index=False)
# df[cols_leves].to_excel("dados_leves.xlsx", index=False)
# print("Exportado!")

## 8. Espaço Livre

Células em branco para suas análises.